<a href="https://colab.research.google.com/github/git-siddhesh/TEGD_thirdai_hackathon/blob/main/Thirdai_gdrive_engine_TEGD.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_directory = '/content/drive/MyDrive'
import os


In [ ]:
loc_map_dict = dict()

## Fetch All the Files

In [ ]:
# Walk through all directories and subdirectories
from collections import defaultdict

all_files = defaultdict(list)

for dirpath, dirnames, filenames in os.walk(root_directory):
    # print(f"Current directory: {dirpath}")

    # List the files in the current directory
    for filename in filenames:
        file_path = os.path.join(dirpath, filename)
        # print(f"File: {file_path}")
        file_type = file_path.split('.')[-1]
        all_files[file_type].append(file_path)


In [ ]:
all_files.keys()

dict_keys(['pdf', 'docx', 'zip', 'gdoc', 'jpeg', 'ipynb', 'jpg', 'm', 'gslides', 'webp', 'xlsx', 'py', 'pptx', 'csv', 'gsheet', 'fig', 'wav', 'txt', 'pkl', 'mp3'])

In [ ]:
for file_ext in ['pdf', 'mp3', 'docx']:
    print(f"file type: {file_ext} has {len(all_files[file_ext])} files")

file type: pdf has 17 files
file type: mp3 has 23 files
file type: docx has 14 files


### Extract the text from mp3 file

#### Gather the requirements

In [ ]:
mp3_train = False
print("Do you want to fetch and train mp3 files too?")
print("It will take a lot of time. We can skip it for now")
# if 'y' in input("Press 'y' to extract audio files else press 'n'").lower():
#   mp3_train = True

Do you want to fetch and train mp3 files too?
It will take a lot of time. We can skip it for now


In [ ]:
mp3_train


False

In [ ]:
if mp3_train:
  !git clone https://github.com/ggerganov/whisper.cpp.git
  %cd /content/whisper.cpp/models
  !bash download-ggml-model.sh base.en
  %cd ..
  !make
  !pip install pydub
  !apt install ffmpeg
  from pydub import AudioSegment

else:
  print("You need to install all the dependencies later")

You need to install all the dependencies later


```python  

!git clone https://github.com/ggerganov/whisper.cpp.git  
%cd /content/whisper.cpp/models  
!bash download-ggml-model.sh base.en  
%cd ..  
!make   
```

```python  
!pip install pydub  
!apt install ffmpeg  
```

#### UTILS functions

> Imports

In [ ]:
import os
import csv
import subprocess

> mp3 -> wav

In [ ]:
def convert_to_wav(file_path, audio_log_dir):
    # Split the file path into base and extension
    base, ext = os.path.splitext(file_path)
    # Only process files that aren't already in wav format
    if ext.lower() != ".wav":
        # Load the audio file
        audio = AudioSegment.from_file(file_path)
        # Set frame rate to 16 kHz
        audio = audio.set_frame_rate(16000)
        # Save as wav
        wav_file_path = audio_log_dir + base.split('/')[-1] + ".wav"
        audio.export(wav_file_path, format="wav")
        print(f"Saved as {wav_file_path}")
    else:
        print(f"File is already in wav format: {file_path}")
    return wav_file_path

> wav -> text (using whisper)

In [ ]:
def transcript_wav_totext_using_whisper(wav_file_path):
    base, ext = os.path.splitext(wav_file_path)
    command = f"./main -f '{wav_file_path}' >  {base}.txt"
    print(command)
    process = subprocess.run(command, shell=True, check=True)
    text_file_path = f'{base}.txt'
    return text_file_path

> text -> csv

In [ ]:
def generate_transcription_csv(text_file_path):
    csv_file_path = text_file_path.replace('.txt', '.csv')
    with open(text_file_path, 'r') as txt_file, open(csv_file_path, 'w') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['time_stamp', 'transcription'])  # Writing header
        for line in txt_file:
            line = line.strip()
            # Skip empty lines
            if line == '':
                continue
            # Split the line into timestamp and transcription
            time_stamp, transcription = line.split(']', 1)
            # Remove the leading '[' from the timestamp
            time_stamp = time_stamp[1:]
            # Remove leading and trailing spaces from the transcription
            transcription = transcription.strip()
            # Skip lines where the transcription is enclosed in square brackets
            if not (transcription.startswith('[') and transcription.endswith(']')):
                # Write the data to the CSV file
                csv_writer.writerow([time_stamp, transcription])
    print(f"CSV file has been saved at: {csv_file_path}")
    return csv_file_path

#### **Final** call: mp3 -> *csv*

In [ ]:
def begin(file_path, audio_log_dir):
    wav_file_path = convert_to_wav(file_path, audio_log_dir)
    #################### Map The original path of the mp3 file to csv file ##############
    text_file_path = transcript_wav_totext_using_whisper(wav_file_path)
    csv_file_path = generate_transcription_csv(text_file_path)
    loc_map_dict[csv_file_path] = file_path
    print(f'file_path {file_path}')
    print(f'wav_file_path {wav_file_path}')
    print(f'text_file_path {text_file_path}\n')
    print(f"File {file_path} converted to csv {text_file_path}")

In [ ]:

if mp3_train:
  print(root_directory)
  os.chdir(root_directory)
  !pwd
  audio_log_dir = root_directory+"/log_dir/audio_csv/"
  if not os.path.exists(audio_log_dir):
      print("Create audio log dir")
      os.makedirs('log_dir/audio_csv')

  else:
      print("path already exist",audio_log_dir)

  os.chdir('/content/whisper.cpp/')
  for mp3_file in all_files['mp3']:
      begin(mp3_file, audio_log_dir)
      break



In [ ]:
loc_map_dict

{}

## Train Neural DB

> imports


```python
!pip3 install thirdai --upgrade
!pip3 install thirdai[neural_db]
!pip3 install langchain --upgrade
!pip3 install openai --upgrade
!pip3 install paper-qa --upgrade
```

In [ ]:
!pip3 install thirdai --upgrade -q
!pip3 install thirdai[neural_db] -q
!pip3 install langchain --upgrade -q
!pip3 install openai --upgrade -q
!pip3 install paper-qa --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 65.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 50.5 MB/s eta 0:00:0

In [ ]:
import pandas as pd
from thirdai import licensing, neural_db as ndb
licensing.deactivate()
licensing.activate("1FB7DD-CAC3EC-832A67-84208D-C4E39E-V3")

In [ ]:
db = ndb.NeuralDB(user_id="siddhesh_saiKrishna")


In [ ]:
os.chdir("/content")
os.getcwd()

'/content'

In [ ]:
# Set up a cache directory
import os
if not os.path.isdir("bazaar_cache"):
    os.mkdir("bazaar_cache")
from pathlib import Path
from thirdai.neural_db import Bazaar
bazaar = Bazaar(cache_dir=Path("bazaar_cache"))

In [ ]:
bazaar.fetch() # Optional arg filter="model name" to filter by model name.

In [ ]:
print(bazaar.list_model_names())

['Contract Review', 'Finance QnA', 'General QnA']


In [ ]:
db = bazaar.get_model("General QnA")

## Insert files to N-DB

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### PDFS

In [ ]:
insertable_docs = []
pdf_files = all_files['pdf']

for file in pdf_files:
    try:
        pdf_doc = ndb.PDF(file)
        insertable_docs.append(pdf_doc)
        print(f'{file} done')
    except:
        pass

/content/drive/MyDrive/22210045_A1.pdf done
/content/drive/MyDrive/22210045_AI.pdf done
/content/drive/MyDrive/22210045_expert.pdf done
/content/drive/MyDrive/22210045_general.pdf done
/content/drive/MyDrive/Edited - exam_1_siddhesh_22210045.pdf done
/content/drive/MyDrive/CCL Tennis Teams List - Sheet2.pdf done
/content/drive/MyDrive/PARAM Ananta User Guide 1.1.pdf done
/content/drive/MyDrive/Tut_1_SiddhiRahpurohit_.pdf done
/content/drive/MyDrive/Classroom/HS 201 World Civilisations Journal Writing/22210045_journal3.pdf done
/content/drive/MyDrive/Classroom/HS 201 World Civilisations Journal Writing/22210045_journal_4.pdf done
/content/drive/MyDrive/Classroom/Computational Neuroscience 2023/exam_1_siddhesh_22210045.pdf done
/content/drive/MyDrive/Classroom/Computational Neuroscience 2023/22210045_project_comp_neuro__Copy_ (2).pdf done
/content/drive/MyDrive/Classroom/Computational Neuroscience 2023/Edited - exam_1_siddhesh_22210045.pdf done
/content/drive/MyDrive/Classroom/Computatio

#### docx

In [ ]:
doc_files = all_files['docx']

for file in doc_files:
    try:
        doc = ndb.DOCX(file)
        insertable_docs.append(doc)
        print(f'{file} done')
    except:
        pass

/content/drive/MyDrive/22210045_AI.docx done
/content/drive/MyDrive/22210045_Assignment1.docx done
/content/drive/MyDrive/22210045_A1.docx done
/content/drive/MyDrive/22210045_A2.docx done
/content/drive/MyDrive/22210045_Assignment_3 (1).docx done
/content/drive/MyDrive/22210045_Assignment_3.docx done
/content/drive/MyDrive/HSS_WCC_2 (1).docx done
/content/drive/MyDrive/HSS_WCC_2.docx done
/content/drive/MyDrive/Do's and Don't AC in hostels.docx done
/content/drive/MyDrive/Classroom/HS 201 World Civilisations Journal Writing/HSS_history.docx done
/content/drive/MyDrive/Classroom/HS 201 World Civilisations Journal Writing/HSS_WCC_2.docx done
/content/drive/MyDrive/Classroom/Plag sid 1/HSS_WCC_2.docx done
/content/drive/MyDrive/Classroom/check-suraj/HSS_WCC_2 (1).docx done
/content/drive/MyDrive/Classroom/check-suraj/HSS_WCC_2.docx done


In [ ]:
loc_map_dict.keys()


dict_keys([])

#### music

In [ ]:
import pandas as pd




In [ ]:
if mp3_train:
    audio_files = [audio_log_dir+file for file in os.listdir(audio_log_dir) if file.endswith('.csv')]
    for file in audio_files:
        df = pd.read_csv(file)
        # Add the new column with values from 0 to n-1
        df['DOC_ID'] = range(len(df))
        # Save the updated DataFrame back to the CSV file
        df.to_csv(file, index=False)
    for file in audio_files:
        try:
            csv_doc = ndb.CSV(
                path=file,
                id_column="DOC_ID",
                strong_columns=["transcription"],
                weak_columns=["time_stamp"],
                reference_columns=["time_stamp"])
            insertable_docs.append(csv_doc)
        except:
            pass

### Insert into database

In [ ]:
source_ids = db.insert(insertable_docs, train=False)


# Search

In [ ]:
query = input("Enter prompt: ")
search_results = db.search(
    # query="what is the termination period",
    query=query,
    top_k=3,
    on_error=lambda error_msg: print(f"Error! {error_msg}"))

for i, result in enumerate(search_results):
    print(f"------------------ TOP {i+1} result -----------------------")
    print(result.text)
    # print(result.context(radius=1))
    print("______ SOURCE _________")
    if loc_map_dict.get(result.source, None) == None:
        print(result.source)
    else:
        print(loc_map_dict.get(result.source, None))
    # print(result.metadata)
    print("--------------------------------------------------")

Enter prompt: System Architecture and Configuration 8 S
------------------ TOP 1 result -----------------------
PARAM Ananta - User's Manual Page | 8 System Architecture and Configuration System Hardware Specifications PARAM Ananta system is based on processor Intel Xeon Platinum 8268 and Intel Xeon Gold 6248 and with total peak performance of 838 TFLOPS. The cluster consists of compute nodes connected with BullSequana XH2000 HDR 100 InfiniBand interconnect network. The system uses the Lustre parallel file system.
______ SOURCE _________
/content/drive/MyDrive/PARAM Ananta User Guide 1.1.pdf
--------------------------------------------------
------------------ TOP 2 result -----------------------
__COMPARISION WITH OTHER WORK__ Comparison of architecture A comparison of architecture and input choices across studies using the publicly available DEAP [56] dataset. The table shows study effectiveness as accuracy increases moving down the table.
______ SOURCE _________
/content/drive/MyDri